In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Loading modules

In [9]:
import mbuild as mb
import foyer
from foyer import Forcefield

import mbuild.formats.charmm_writer as mf_charmm

import mosdef_cp2k_writer


import parmed as pmd
from constrainmol import ConstrainedMolecule
import unyt as u

from mosdef_cp2kmcpy.mc import MC
import setter
warnings.filterwarnings("ignore", category=DeprecationWarning)
 

## The forcefield file

In [10]:
!cat ammonia_ff.xml

<ForceField>
    <AtomTypes>
        <Type name="N" def="[N;X3](H)(H)(H)"
              class="NH3" element="N" mass=" 14.0067" desc="N in NH3"
              doi="random" />
        <Type name="H" def="[H;X1](N)"
              class="NH3" element="H" mass=" 1.008" desc="H in NH3"
              doi="random" />
    </AtomTypes>
    <HarmonicBondForce>
        <Bond class1="NH3" class2="NH3" length="0.1008" k="350500"/>
    </HarmonicBondForce>
    <HarmonicAngleForce>
	<Angle class1="NH3" class2="NH3" class3="NH3" angle="1.618" k="369.44"/>
    </HarmonicAngleForce>
    <RBTorsionForce>
    </RBTorsionForce>
    <NonbondedForce coulomb14scale="0" lj14scale="0">
        <Atom type="N" charge="-0.2376" sigma="0.32973" epsilon="0.30491436"/>
	<Atom type="H" charge="0.0792" sigma="0.25" epsilon="0.3"/>
    </NonbondedForce>
</ForceField>

## Defining the molecule



In [11]:

ammonia_res_name = 'NH3'
FF_file_ammonia = 'ammonia_ff.xml'
ammonia_FF = Forcefield(forcefield_files=FF_file_ammonia)
ammonia_mb = mb.load('N', smiles=True)

ammonia_mb.name = ammonia_res_name

ammonia_typed=ammonia_FF.apply(ammonia_mb)



constrain_mol = ConstrainedMolecule(ammonia_typed)
constrain_mol.solve()
ammonia_optimized_mb=mb.clone(ammonia_mb)
ammonia_optimized_mb.xyz=constrain_mol.xyz/10
print(ammonia_optimized_mb.xyz)

/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/foyer/forcefield.py:448: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/foyer/forcefield.py:460: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/mnt/d/software/MC-workflow/mbuild/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule N	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/rs/anaconda3/envs/mosdef_cp2k/lib/python3.8/site-packages/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 1, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)


[[ 0.10649249 -0.00560326  0.000344  ]
 [ 0.2071696  -0.00625679 -0.00458871]
 [ 0.07527927 -0.09244943 -0.04020372]
 [ 0.075279    0.06768258 -0.06142609]]


In [14]:
ammonia_optimized_mb.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Creating .psf (Topology) and .inp (CHARMM potential file) using mosdef_charmm_writer


In [16]:
#random box to be used in mosdef_charmm_writer
ammonia_box_bias  = mb.fill_box(compound=[ammonia_optimized_mb ],
                             n_compounds=[1] ,
                            box=[1.0, 1.0, 1.0] )

### mosdef_charmm_writer

In [17]:
FF_Dict = {ammonia_optimized_mb.name:FF_file_ammonia }

residues_List = [ammonia_optimized_mb.name ]




ammonia_box_bias.save("ammonia_bias_coord.xyz",overwrite=True)

print('Running: GOMC FF file, and the psf and pdb files for the biasing potential file')
mf_charmm.charmm_psf_psb_FF(ammonia_box_bias ,
                            'ammonia_bias',
                            FF_filename ="ammonia_charmm_bias" ,
                            forcefield_selection = FF_Dict,
                            residues= residues_List ,
                            bead_to_atom_name_dict = {},
                            fix_residue = None,
                            reorder_res_in_pdb_psf = False
                            )

! rm *.pdb *.xyz

Running: GOMC FF file, and the psf and pdb files for the biasing potential file
write_gomcdata: forcefield_selection = {'NH3': 'ammonia_ff.xml'}, residues = ['NH3']
******************************

GOMC FF writing each residues FF as a group for structure_0
forcefield type from compound = {'NH3': 'ammonia_ff.xml'}
coulomb14scale from compound = {'NH3': 0.0}
lj14scale from compound = {'NH3': 0.0}
unique_types = ['H_NH3', 'N_NH3']
No urey bradley terms detected, will use angle_style harmonic
******************************

writing the GOMC force field file 
NBFIX_Mixing not used or no mixing used for the non-bonded potentials out
forcefield_dict = {1: 'N_NH3', 0: 'H_NH3'}
******************************

write_charmm_psf file is running
write_charmm_psf: forcefield_selection = {'NH3': 'ammonia_ff.xml'}, residues = ['NH3']
******************************

No urey bradley terms detected
******************************

write_charmm_pdb file is running
write_charmm_pdb: residues == ['NH3']
fix_

### Now we have the .psf (topology) file and .inp (charmm potential) file

## Creating CP2K MC simulation

In [18]:
molecule_list=[ammonia_optimized_mb]
box=mb.box.Box(lengths=[1,1,1])
#Comment why pressure is needed
q=MC(molecule_list=molecule_list,n_box=2,n_molecules_each_box=[[49],[1]], box_list=[box,box],cutoff=200,functional='PBE',
     basis_set={'N':'DZVP-MOLOPT-GTH','H':'DZVP-MOLOPT-GTH'}, periodicity=['XYZ']*2,ensemble='GEMC_NVT',seed=1,project_name='NH3_NVT_GEMC',restart='FALSE',pressure=1*u.bar)

In [19]:
q.mc_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
n_steps not specified, set as 1000
n_ff_moves not specified, set as 8
nswapmoves not specified, set as 640, will be ignore if n_box<2
output trajectory format set as XYZ
input_filename not specified, set as ['GEMC_NVT_box1.inp', 'GEMC_NVT_box2.inp']
output_filename not specified, set as NH3_NVT_GEMC_mc_output.out
temperature not defined, set as 298 K
You can change default settings in setter.mc_files


In [20]:

q.topology_filename=['ammonia_bias.psf']

# move_probabilities=[pmavbmc,pmcltrans,pmhmc,pmswap,pmtraion,pmtrans,pmvolume]
#volume moves = PMVOLUME
# swap moves = PMSWAP - PMVOLUME
# AVBMC moves = PMAVBMC - PMSWAP - PMVOLUME
# an “inner” move = 1.0 - (PMAVBMC + PMSWAP + PMVOLUME)
#conformational changes = “inner” move percentage × PMTRAION
# molecular translation = “inner” move percentage × (PMTRANS - PMTRAION)
# molecular rotation = “inner” move percentage × (1.0 - PMTRANS - PMTRAION)

q.move_probabilities=[0,0.0,0,0.4,0.5,0.75,0.05]



# mol_probabilities=[[PMAVBMC_MOL,PMSWAP_MOL , PMTRAION_MOL, PMTRANS_MOL,PMROT_MOL],[]]
q.mol_probabilities=[[[1],[1],[1],[1],[1]],[[1],[1],[1],[1],[1]]]



#avbmc probabilities=[[AVBMC_ATOM,AVBMC_RMIN,AVBMC_RMAX,PBIAS],[]]
q.avbmc_probabilities=[[[1],[1],[1],[1]],[[1],[1],[1],[1]]]

q.charmm_potential_file='ammonia_charmm_bias.inp'

In [21]:
setter.mc_files(q)

/mnt/d/software/MC-workflow/mbuild/mbuild/packing.py:689: UserWarning: Packmol finished with imperfect packing. Using the .xyz_FORCED file instead. This may not be a sufficient packing result.
  warnings.warn(msg)


MC initial structure saved as ['NH3_NVT_GEMC_box1_initial.xyz', 'NH3_NVT_GEMC_box2_initial.xyz']
MC input file saved as GEMC_NVT_box1.inp
MC input file saved as GEMC_NVT_box2.inp


In [22]:
!ls

GEMC_NVT_box1.inp	       changeLog.out
GEMC_NVT_box2.inp	       dinitrogen_ff.xml
NH3_NVT_GEMC_box1_initial.xyz  log.txt
NH3_NVT_GEMC_box2_initial.xyz  md-changeLog.out
Untitled.ipynb		       nitrogen_bias.psf
__pycache__		       nitrogen_charmm_bias.inp
ammonia_bias.psf	       old.ipynb
ammonia_charmm_bias.inp        setter.py
ammonia_ff.xml		       trajectories
bias_template.inp


In [24]:
!cp2k.popt -i GEMC_NVT_box1.inp -i GEMC_NVT_box2.inp

 DBCSR| Multiplication driver                                               BLAS
 DBCSR| Multrec recursion limit                                              512
 DBCSR| Multiplication stack size                                           1000
 DBCSR| Maximum elements for images                                    UNLIMITED
 DBCSR| Multiplicative factor virtual images                                   1
 DBCSR| Multiplication size stacks                                             3
 DBCSR| Number of 3D layers                                               SINGLE
 DBCSR| Use MPI memory allocation                                              T
 DBCSR| Use RMA algorithm                                                      F
 DBCSR| Use Communication thread                                               T
 DBCSR| Communication thread load                                             87


  **** **** ******  **  PROGRAM STARTED AT               2021-02-18 12:09:00.631
 ***** ** ***  *** **   PR


 *******************************************************************************
 *******************************************************************************
 **                                                                           **
 **     #####                         ##              ##                      **
 **    ##   ##            ##          ##              ##                      **
 **   ##     ##                       ##            ######                    **
 **   ##     ##  ##   ##  ##   #####  ##  ##   ####   ##    #####    #####    **
 **   ##     ##  ##   ##  ##  ##      ## ##   ##      ##   ##   ##  ##   ##   **
 **   ##  ## ##  ##   ##  ##  ##      ####     ###    ##   ######   ######    **
 **    ##  ###   ##   ##  ##  ##      ## ##      ##   ##   ##       ##        **
 **     #######   #####   ##   #####  ##  ##  ####    ##    #####   ##        **
 **           ##                                                    ##        **
 **                        


 Number of electrons:                                                          8
 Number of occupied orbitals:                                                  4
 Number of molecular orbitals:                                                 4

 Number of orbital functions:                                                 28
 Number of independent orbital functions:                                     28

 Extrapolation method: initial_guess

 *** WARNING in qs_initial_guess.F:263 :: User requested to restart the    ***
 *** wavefunction from the file named: NH3_NVT_GEMC-RESTART.wfn. This file ***
 *** does not exist. Please check the existence of the file or change      ***
 *** properly the value of the keyword WFN_RESTART_FILE_NAME. Calculation  ***
 *** continues using ATOMIC GUESS.                                         ***



 SCF WAVEFUNCTION OPTIMIZATION

  ----------------------------------- OT ---------------------------------------
  Minimizer      : DIIS                : di